In [1]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas, xgboost, numpy, textblob, string, re, nltk
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel('TEC_ML_ACT.xlsx')

In [3]:
import openpyxl

wb = openpyxl.load_workbook('noChinese1111ValueCounts.xlsx')
sheet = wb.active
lst=[]
for i in range(3,52):
    lst.append(sheet.cell(i, 1).value)

In [5]:
data1 = data.loc[data['Subject']=='Call']
for i in lst:
    data1 = data1.append(data.loc[data['Subject']==str(i)])

In [6]:
data1.to_excel('trainDFwithChinese.xlsx')

In [7]:
newdata = data
newdata.drop(newdata.loc[newdata['Subject']=='Call'].index, inplace=True)
for i in lst:
    newdata.drop(newdata.loc[newdata['Subject']==str(i)].index, inplace=True)

In [8]:
len(newdata)+len(data1)

305409

In [9]:
newdata.to_excel('testDFwithChinese.xlsx')

In [10]:
trainDF = pandas.read_excel("trainDFwithChinese.xlsx")
trainDF.head()

,Unnamed: 0,Activity_Id,Type_of_Activity__c,Activity_Type_SFDC__c,Owner_Country__c,Subject,Description
0,0,00T6F00002bzw4TUAQ,Task,Call,Thailand,Call,Call
1,2,00T6F00002jKce3UAC,Task,Call,China,Call,Call
2,3,00T6F00002lQpA0UAK,Task,Call,Thailand,Call,Call
3,5,00T6F00002p1Gg6UAE,Task,Call,Thailand,Call,Call
4,6,00T6F00002Sa7D9UAJ,Task,Call,Thailand,Call,Call


In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
# Function to preprocess the reviews data
def preprocess_review_data(data,name):
    # Proprocessing the data
    data[name]=data[name].str.lower()
    # Code to remove the Hashtags from the text
    data[name]=data[name].apply(lambda x:re.sub(r'\B#\S+','',str(x)))
    # Code to remove the links from the text
    data[name]=data[name].apply(lambda x:re.sub(r"http\S+", "", str(x)))
    # Code to remove the Special characters from the text 
    data[name]=data[name].apply(lambda x:' '.join(re.findall(r'\w+', str(x))))
    # Code to substitute the multiple spaces with single spaces
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
    # Code to remove all the single characters in the text
    data[name]=data[name].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', '', x))
    # Remove the twitter handlers
    data[name]=data[name].apply(lambda x:re.sub('@[^\s]+','',x))
    
    # Function to tokenize and remove the stopwords    
def tokenize(data,name):
      
    def getting(sen):
        example_sent = sen

        word_tokens = word_tokenize(example_sent) 

        return word_tokens
    
    x=[]
    for i in data[name].values:
        x.append(getting(i))
    data[name]=x
# Making a function to lemmatize all the words
lemmatizer = WordNetLemmatizer() 
def lemmatize_all(data,name):
    arr=data[name]
    a=[]
    for i in arr:
        b=[]
        for j in i:
            x=lemmatizer.lemmatize(j,pos='a')
            x=lemmatizer.lemmatize(x)
            b.append(x)
        a.append(b)
    data[name]=a
# Function to make it back into a sentence 
def make_sentences(data,name):
    data[name]=data[name].apply(lambda x:' '.join([i+' ' for i in x]))
    # Removing double spaces if created
    data[name]=data[name].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))

In [15]:
# Using the preprocessing function to preprocess the tweet data
preprocess_review_data(trainDF,'Description')
# Using tokenizer and removing the stopwords
tokenize(trainDF,'Description')
# Converting all the texts back to sentences
make_sentences(trainDF,'Description')

In [16]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['Description'], trainDF['Subject'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [17]:
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(trainDF['Subject'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

xtrain_tfidf_ngram_chars.todense()

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:546: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn(


matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.03585922, 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [18]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [19]:
# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("NB, CharLevel Vectors: ", accuracy)

NB, CharLevel Vectors:  0.7612862660558699


In [20]:
# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print("LR, CharLevel Vectors: ", accuracy)

LR, CharLevel Vectors:  0.8061618611335669


C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print("Xgb, CharLevel Vectors: ", accuracy)

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:24:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Xgb, CharLevel Vectors:  0.8176592113536334


In [23]:
testDF = pd.read_excel("testDFwithChinese.xlsx")
testDF = testDF.dropna()
testDF.head()

,Unnamed: 0,Activity_Id,Type_of_Activity__c,Activity_Type_SFDC__c,Owner_Country__c,Subject,Description
0,1,00T6F00002ikD3WUAU,Task,Call,Taiwan,Trouble shooting,Call
1,9,00T6F00002wX19dUAC,Task,Email,Taiwan,NDA discussion,Email
2,14,00T6F00002y8oA3UAI,Task,Call,Thailand,Call - Appoint to meet,Call - appoint to demo
3,15,00T6F00002y8oOVUAY,Task,Email,Thailand,Email - Intro AccuNest,intro accunest paper_x000D_\n- brochure_x000D_...
4,17,00T6F00002yn5OxUAI,Task,Email,Taiwan,NDA Review,Email


In [24]:
# Using the preprocessing function to preprocess the tweet data
preprocess_review_data(testDF,'Description')
# Using tokenizer and removing the stopwords
tokenize(testDF,'Description')
# Converting all the texts back to sentences
make_sentences(testDF,'Description')

In [25]:
test_x = testDF['Description']

In [26]:
# characters level tf-idf
xtest_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(test_x) 

In [27]:
def predict(classifier, feature_vector_train, label, feature_vector_valid):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    testDF["Predictions"] = predictions

# Linear Classifier on Count Vectors
predict(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars, train_y, xtest_tfidf_ngram_chars)
testDF.head()

C:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:36:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,Unnamed: 0,Activity_Id,Type_of_Activity__c,Activity_Type_SFDC__c,Owner_Country__c,Subject,Description,Predictions
0,1,00T6F00002ikD3WUAU,Task,Call,Taiwan,Trouble shooting,call,1
1,9,00T6F00002wX19dUAC,Task,Email,Taiwan,NDA discussion,email,13
2,14,00T6F00002y8oA3UAI,Task,Call,Thailand,Call - Appoint to meet,call appoint to demo,1
3,15,00T6F00002y8oOVUAY,Task,Email,Thailand,Email - Intro AccuNest,intro accunest paper_x000d_ brochure_x000d_ co...,1
4,17,00T6F00002yn5OxUAI,Task,Email,Taiwan,NDA Review,email,13


In [28]:
testDF['label']=encoder.inverse_transform(testDF['Predictions'])
testDF

,Unnamed: 0,Activity_Id,Type_of_Activity__c,Activity_Type_SFDC__c,Owner_Country__c,Subject,Description,Predictions,label
0,1,00T6F00002ikD3WUAU,Task,Call,Taiwan,Trouble shooting,call,1,Call
1,9,00T6F00002wX19dUAC,Task,Email,Taiwan,NDA discussion,email,13,Email
2,14,00T6F00002y8oA3UAI,Task,Call,Thailand,Call - Appoint to meet,call appoint to demo,1,Call
3,15,00T6F00002y8oOVUAY,Task,Email,Thailand,Email - Intro AccuNest,intro accunest paper_x000d_ brochure_x000d_ co...,1,Call
4,17,00T6F00002yn5OxUAI,Task,Email,Taiwan,NDA Review,email,13,Email
...,...,...,...,...,...,...,...,...,...
82746,305388,00U6F00002EOy3GUAT,Event,Visit,Taiwan,拜訪H棟開刀房志豪護理長,visit,47,Visit
82747,305389,00U6F00002EOy3LUAT,Event,Visit,Taiwan,拜訪醫工林康慈先生,visit,47,Visit
82748,305390,00U6F00002EOYfCUAX,Event,Visit,Taiwan,拜訪供應中心芷芸學姊,visit,47,Visit
82749,305391,00U6F00002EOYfXUAX,Event,Visit,Taiwan,拜訪醫工林康慈先生,visit,47,Visit
